In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [64]:
#importando as bibliotecas que serão usadas
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

In [65]:
#dados para treino da IA
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
#dados para teste da IA
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [66]:
#observar os dados para treino
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [67]:
#informações estatísticas cada coluna dos dados de treino
train_df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.0,1460.0,1201.0,1460.0,1460.0,1460.0,1460.0,1460.0,1452.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1379.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0,1460.0
mean,730.5,56.9,70.0,10516.8,6.1,5.6,1971.3,1984.9,103.7,443.6,46.5,567.2,1057.4,1162.6,347.0,5.8,1515.5,0.4,0.1,1.6,0.4,2.9,1.0,6.5,0.6,1978.5,1.8,473.0,94.2,46.7,22.0,3.4,15.1,2.8,43.5,6.3,2007.8,180921.2
std,421.6,42.3,24.3,9981.3,1.4,1.1,30.2,20.6,181.1,456.1,161.3,441.9,438.7,386.6,436.5,48.6,525.5,0.5,0.2,0.6,0.5,0.8,0.2,1.6,0.6,24.7,0.7,213.8,125.3,66.3,61.1,29.3,55.8,40.2,496.1,2.7,1.3,79442.5
min,1.0,20.0,21.0,1300.0,1.0,1.0,1872.0,1950.0,0.0,0.0,0.0,0.0,0.0,334.0,0.0,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2006.0,34900.0
25%,365.8,20.0,59.0,7553.5,5.0,5.0,1954.0,1967.0,0.0,0.0,0.0,223.0,795.8,882.0,0.0,0.0,1129.5,0.0,0.0,1.0,0.0,2.0,1.0,5.0,0.0,1961.0,1.0,334.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,129975.0
50%,730.5,50.0,69.0,9478.5,6.0,5.0,1973.0,1994.0,0.0,383.5,0.0,477.5,991.5,1087.0,0.0,0.0,1464.0,0.0,0.0,2.0,0.0,3.0,1.0,6.0,1.0,1980.0,2.0,480.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,6.0,2008.0,163000.0
75%,1095.2,70.0,80.0,11601.5,7.0,6.0,2000.0,2004.0,166.0,712.2,0.0,808.0,1298.2,1391.2,728.0,0.0,1776.8,1.0,0.0,2.0,1.0,3.0,1.0,7.0,1.0,2002.0,2.0,576.0,168.0,68.0,0.0,0.0,0.0,0.0,0.0,8.0,2009.0,214000.0
max,1460.0,190.0,313.0,215245.0,10.0,9.0,2010.0,2010.0,1600.0,5644.0,1474.0,2336.0,6110.0,4692.0,2065.0,572.0,5642.0,3.0,2.0,3.0,2.0,8.0,3.0,14.0,3.0,2010.0,4.0,1418.0,857.0,547.0,552.0,508.0,480.0,738.0,15500.0,12.0,2010.0,755000.0


In [68]:
#informações de cada coluna dos dados de treino
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [69]:
#vamos remover as colunas que mais possuem valores naN
treino_tratado=train_df.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature','MasVnrType'],axis=1)
teste_tratado=test_df.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature','MasVnrType'],axis=1)

In [70]:
#como ficou o data frame:
treino_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [71]:
# agora vamos substituir todos os naN por -1
treino_tratado_na=treino_tratado.fillna(-1)
teste_tratado_na=teste_tratado.fillna(-1)

In [72]:
#vamos codificar todas as colunas que não possuem valores int ou float
codificador = LabelEncoder()

In [73]:
objList = treino_tratado_na.select_dtypes(include = "object").columns

In [74]:
for feat in objList:
    treino_tratado_na[feat] = codificador.fit_transform(treino_tratado_na[feat].astype(str))
    teste_tratado_na[feat] = codificador.fit_transform(teste_tratado_na[feat].astype(str))

In [75]:
#percebe-se agora que todas as colunas são do tipo int ou float
treino_tratado_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int64  
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int64  
 6   LotShape       1460 non-null   int64  
 7   LandContour    1460 non-null   int64  
 8   Utilities      1460 non-null   int64  
 9   LotConfig      1460 non-null   int64  
 10  LandSlope      1460 non-null   int64  
 11  Neighborhood   1460 non-null   int64  
 12  Condition1     1460 non-null   int64  
 13  Condition2     1460 non-null   int64  
 14  BldgType       1460 non-null   int64  
 15  HouseStyle     1460 non-null   int64  
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [76]:
#vamos observar a correlação entre as colunas
pd.set_option('display.max_columns', None)
display(treino_tratado_na.corr().loc[['SalePrice'],:])

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
SalePrice,-0.0,-0.1,-0.2,0.2,0.3,0.0,-0.3,0.0,-0.0,-0.1,0.1,0.2,0.1,0.0,-0.1,0.2,0.8,-0.1,0.5,0.5,0.2,0.1,0.1,0.1,0.5,-0.6,0.1,0.4,-0.4,0.1,-0.2,-0.0,0.4,0.1,-0.0,0.2,0.6,-0.1,-0.4,0.3,0.2,0.6,0.3,-0.0,0.7,0.2,-0.0,0.6,0.3,0.2,-0.1,-0.6,0.5,0.1,0.5,-0.2,0.3,-0.3,0.6,0.6,0.3,0.3,0.2,0.3,0.3,-0.1,0.0,0.1,0.1,-0.0,0.0,-0.0,-0.1,0.2,1.0


In [77]:
#vamos remover todas as colunas cuja a correlação é nula
treino_tratado_na=treino_tratado_na.drop(['YrSold','MoSold','MiscVal','3SsnPorch','BsmtHalfBath','LowQualFinSF','BsmtFinSF2','BsmtFinType1','Condition2','Utilities','LandContour','Street'],axis=1)
teste_tratado_na=teste_tratado_na.drop(['YrSold','MoSold','MiscVal','3SsnPorch','BsmtHalfBath','LowQualFinSF','BsmtFinSF2','BsmtFinType1','Condition2','Utilities','LandContour','Street'],axis=1)

In [78]:
#preparando os dados para treinar a IA

#x -> dados de treino (sem a coluna 'SalePrice')
#Id foi removido pois não possui apenas identifica o imovel, e não possui nenhuma informaçõa relevante para o treino
x = treino_tratado_na.drop(columns=["SalePrice", "Id"])
#foi removido a coluna "PassengerId" pois ela não é útil para treinar a IA

#y -> gabarito (coluna 'SalePrice')
y = treino_tratado_na.SalePrice

In [79]:
#separa dados de treino e de teste para os modelos
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=1)#,test_size=0.3)

In [80]:
x_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinType2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,PoolArea,SaleType,SaleCondition
6,20,3,75.0,10084,3,4,0,21,2,0,2,8,5,2004,2005,1,1,12,13,186.0,2,4,2,1,4,1,1369,6,317,1686,1,0,1,5,1694,0,1694,1,2,0,3,1,2,7,6,1,2,2004.0,2,2,636,5,5,2,255,57,0,0,0,8,4
807,70,3,144.0,21384,3,4,0,3,2,0,5,5,6,1923,2004,1,1,13,14,0.0,3,4,1,4,4,2,1309,6,15,1324,1,0,1,5,1072,504,1576,2,1,1,3,1,2,6,6,1,2,1923.0,2,2,528,5,5,2,0,312,0,0,0,8,4
955,90,2,82.0,7136,0,4,0,6,2,2,5,6,6,1946,1950,1,1,8,8,423.0,3,4,1,3,4,4,484,6,495,979,1,4,0,2,979,979,1958,0,2,0,4,2,3,8,6,0,2,1946.0,3,2,492,5,5,2,0,0,0,0,0,8,4
1040,20,3,88.0,13125,3,0,0,19,2,0,2,5,4,1957,2000,1,1,13,14,67.0,3,4,1,4,4,4,168,2,284,1134,1,0,1,5,1803,0,1803,1,2,0,3,1,3,8,0,1,2,1957.0,2,2,484,5,5,2,0,0,0,0,0,8,4
701,20,3,80.0,9600,3,4,0,14,2,0,2,7,5,1969,1969,3,1,6,6,168.0,3,4,1,4,4,4,0,6,1164,1164,1,4,1,5,1164,0,1164,0,1,1,3,1,3,6,6,0,2,1969.0,3,2,528,5,5,2,0,0,0,0,0,0,4


Vamos testar três modelos de algotítimos de inteligência artificial para regressão linear: Linear Regression, Regression Tree e KNN Regressor

In [81]:
from sklearn.linear_model import LinearRegression

In [82]:
#treino
reg_rl = LinearRegression().fit(x_train,y_train)

In [83]:
#previsão
y_rl = reg_rl.predict(x_test)

In [84]:
from sklearn import tree

In [85]:
#treino
reg_ar = tree.DecisionTreeRegressor(random_state=1).fit(x_train,y_train)

In [86]:
#previsão
y_ar = reg_ar.predict(x_test)

In [87]:
from sklearn.neighbors import KNeighborsRegressor

In [88]:
#treino
reg_knn = KNeighborsRegressor(n_neighbors=2).fit(x_train,y_train)

In [89]:
#previsão
y_knn = reg_knn.predict(x_test)

In [90]:
#vamos avaliar o erro das previsões de cada modelo, avaliando o erro absoluto e o erro quadrático
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [97]:
print('Erro absoluto Regressão Linear: ',mean_absolute_error(y_test,y_rl))
print('Erro quadrático Regressão Linear: ',mean_squared_error(y_test,y_rl))

Erro absoluto Regressão Linear:  19554.381925514383
Erro quadrático Regressão Linear:  1140295688.759287


In [95]:
print('Erro absoluto Arvore de Decisão: ',mean_absolute_error(y_test,y_ar))
print('Erro quadrático Arvore de Decisão: ',mean_squared_error(y_test,y_ar))

Erro absoluto Arvore de Decisão:  26031.68493150685
Erro quadrático Arvore de Decisão:  1465688297.7232876


In [96]:
print('Erro absoluto KNN: ',mean_absolute_error(y_test,y_knn))
print('Erro quadrático KNN: ',mean_squared_error(y_test,y_knn))

Erro absoluto KNN:  30219.930136986302
Erro quadrático KNN:  2556392136.2856164


In [98]:
#percebe-se que o modelo de menor erro é o LinearRegression

Agora vamos usar o melhor modelo para prever os dados test.csv

In [99]:
#removendo a coluna Id
x_sub = teste_tratado_na.drop(columns=["Id"])

In [100]:
#usando o modelo LinearRegression para prever o SalePrice
x_sub_rl = reg_rl.predict(x_sub)

In [101]:
#constrindo o data frame para a subimissão ao Kaggle
Submission=pd.DataFrame({'Id':teste_tratado_na['Id'],'SalePrice':x_sub_rl})

In [102]:
Submission.head()

,Id,SalePrice
0,1461,102803.8
1,1462,155754.4
2,1463,159392.9
3,1464,184746.2
4,1465,184724.1


In [103]:
Submission.shape

(1459, 2)

In [104]:
Submission.to_csv('submission.csv', index=False)

Minha subimissão obteve pontuação de 0.18763 no Kaggle (A pontuação vai de 1 a 0, em que 0 é a melhor pontuação).